In [1]:
import findspark
findspark.init()

import pyspark

from pyspark.sql import SparkSession

trab_des_data = 'C:/Users/User/entornos/eltrabajodeseado/analisis_notebooks/usuarios_candidatos_2.csv'



In [2]:
# create a SparkSession: note this step was left out of the screencast
spark = SparkSession.builder \
    .master("local") \
    .appName("Clientes") \
    .getOrCreate()

In [38]:
df = spark.read.csv(trab_des_data, header=True, inferSchema=True)

df.printSchema()


root
 |-- id: string (nullable = true)
 |-- password1: string (nullable = true)
 |-- password2: string (nullable = true)
 |-- email: string (nullable = true)
 |-- Nombres: string (nullable = true)
 |-- Apellidos: string (nullable = true)
 |-- tipo_de_documento: string (nullable = true)
 |-- documento: string (nullable = true)
 |-- celular: string (nullable = true)
 |-- nivel_de_estudios: string (nullable = true)
 |-- años_de_experiencia: string (nullable = true)
 |-- ciudad_de_residencia: string (nullable = true)
 |-- expectativa_salarial: string (nullable = true)
 |-- perfil_profesional: string (nullable = true)
 |-- genero: string (nullable = true)
 |-- estado_civil: string (nullable = true)
 |-- fecha_de_nacimiento: string (nullable = true)
 |-- direccion_de_residencia: string (nullable = true)
 |-- disponibilidad_para_viajar: string (nullable = true)
 |-- disponibilidad_para_cambiar_de_residencia: string (nullable = true)
 |-- is_active: string (nullable = true)
 |-- fecha_de_regis

In [39]:
print(df.count())

df = df.distinct()

print(df.count())

45211
40929


In [54]:
from pyspark.sql.functions import udf
import re

def es_correo_valido(correo):
    expresion_regular = r"(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|\"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*\")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"
    if re.match(expresion_regular, correo) is not None:
        return correo


validoMail = udf(lambda x: es_correo_valido(x))
df.filter(validoMail(df.email)==df.email).select(['id', 'email', 'Nombres', 'Apellidos', 'tipo_de_documento']).show(100)


+-----+--------------------+------------------+--------------------+-----------------+
|   id|               email|           Nombres|           Apellidos|tipo_de_documento|
+-----+--------------------+------------------+--------------------+-----------------+
| 4464|  silvama7@gmail.com|           Maribel|        Valero Silva|                1|
| 4521| ch.arcila@gmail.com|     Carmen Helena|    Arcila Rodríguez|                1|
| 4535|amontan1@hotmail.com|            Alvaro|   Montaño Gualteros|                1|
| 5620| arlepat@hotmail.com| Arleydis Patricia|       Mejia Alarcón|                1|
| 6359|mcmorajimenez@gma...|    Martha Cecilia|        Mora Jiménez|                1|
| 6614|cynthiahernandez....|  Cynthia Berenice|    Hernandez Vargas|                1|
| 6661|corredorlibys@gma...|             Libys|     Corredor Cortes|                1|
| 6793|novoag22@hotmail.com|     Andres Felipe|        Novoa Galvez|                1|
| 6953|soleyclarena@hotm...|           Clar

In [52]:
df.filter('id' > 0).df.select(['id','password1','email', 'Nombres', 'Apellidos']).show()

TypeError: '>' not supported between instances of 'str' and 'int'

In [ ]:
lista_indices_usuarios = frame_usuarios['id'].values
i =1
candidatos = []

for indice in lista_indices_usuarios:
    print(indice, end=', ')
    
    campos = dict(zip(frame_usuarios.columns, frame_usuarios.loc[frame_usuarios['id']==indice].values[0]))
    campos['perfil_laboral'] = json.loads(campos['perfil_laboral'])
    del campos["id"]
    dic_candi = {"model": "cuentas.candidatos", "pk": i, 
        'fields': campos}
    candidatos.append(dic_candi)

    
    i += 1
    
    #if i > 20:
    #    break



### Las siguientes celdas solo sirven como repositorio, ya fueron ejecutadas y su resultado actualizo el dataframe y json

In [ ]:
lista_indices_usuarios = frame_usuarios['id'].values
i = 1

for indice in lista_indices_usuarios:
    var = frame_usuarios.loc[frame_usuarios['id']==indice].values
    print(var, end=', ')
    
    i += 1
    #if i > 1:
    #    break

In [ ]:
import datetime

def valida_fecha(var):
    try:
        fecha = datetime.datetime(year=int(var[-4:]),month=int(var[3:5]),day=int(var[0:2]))
        es_valida = True
    except:
        es_valida = False
        
    return es_valida


lista_indices_usuarios = frame_usuarios['id'].values
i = 1

for indice in lista_indices_usuarios:
    var = frame_usuarios.loc[frame_usuarios['id']==indice]['fecha_de_nacimiento'].values[0]
    fecha_nueva = var[-2:]+'/'+var[5:7]+'/'+var[0:4]
    
    var1 = frame_usuarios.loc[frame_usuarios['id']==indice]['fecha_de_registro'].values[0]
    fecha_nueva1 = var1[-2:]+'/'+var1[5:7]+'/'+var1[0:4]

    if valida_fecha(fecha_nueva):
        frame_usuarios.loc[frame_usuarios['id']==indice, ['fecha_de_nacimiento']] = fecha_nueva
    else:
        frame_usuarios.loc[frame_usuarios['id']==indice, ['fecha_de_nacimiento']] = '01/01/2019'

    if valida_fecha(fecha_nueva1):
        frame_usuarios.loc[frame_usuarios['id']==indice, ['fecha_de_registro']] = fecha_nueva1
    else:
        frame_usuarios.loc[frame_usuarios['id']==indice, ['fecha_de_registro']] = '19/01/2019'
    print(fecha_nueva, fecha_nueva1)

In [ ]:
faltan = [ 'Nombres', 'Apellidos', 'celular', 'nivel_de_estudios', 'perfil_profesional', 'genero',
          'estado_civil', 'direccion_de_residencia']

for falta in faltan:
    frame_usuarios.loc[frame_usuarios[falta].isnull(), falta] = ''

In [ ]:
premion = [ 'dianadelgadoing@gmail.com',   'wilmerandreszerpa@hotmail.com', 'rodriguezaguilarjc@hotmail.com',
           'bibanez04@gmail.com', 'carcinie@gmail.com', 'luisarcinie1@gmail.com', 'valentincc94m@gmail.com' ]

frame_usuarios['is_active'] = False

for premi in premion:
    frame_usuarios.loc[frame_usuarios['email']==premi, ['is_active']] = True

In [ ]:
lista_indices_usuarios = frame_usuarios['id'].values
i = 1

for indice in lista_indices_usuarios:
    var = frame_usuarios.loc[frame_usuarios['id']==indice]['fecha_de_nacimiento'].values[0]
    fecha_nueva = var[-4:]+'/'+var[3:5]+'/'+var[0:2]
    
    var1 = frame_usuarios.loc[frame_usuarios['id']==indice]['fecha_de_registro'].values[0]
    fecha_nueva1 = var1[-4:]+'/'+var1[3:5]+'/'+var1[0:2]

    frame_usuarios.loc[frame_usuarios['id']==indice, ['fecha_de_nacimiento']] = fecha_nueva
    frame_usuarios.loc[frame_usuarios['id']==indice, ['fecha_de_registro']] = fecha_nueva1
    
    print(fecha_nueva, fecha_nueva1)

In [ ]:
var = '29/01/2019'
var[-4:]
var[0:2]
var[3:5]

In [ ]:
frame_usuarios.to_csv('usuarios_candidatos_2.csv',index=False)
frame_usuarios.info()

In [ ]:
frame_usuarios.loc[frame_usuarios['is_active']==True]

In [ ]:

           'bibanez04@gmail.com', 

In [ ]:
user = int(frame_usuarios.loc[frame_usuarios['id']==23973].index[0])
user

In [ ]:
frame_usuarios = frame_usuarios.loc[frame_usuarios['is_active']]

In [ ]:
filas = frame_usuarios['id'].values

for fila in filas:
    temp = frame_usuarios.loc[frame_usuarios['id']==fila]
    
    for columna in temp.columns:
        print(f"{columna}, {temp[columna].values}")
    print('=================')

In [ ]:
frame_usuarios

In [ ]:
filas